In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [1]:
cd gdrive/My Drive/Voice_Cloning/gantts

/content/gdrive/My Drive/Voice_Cloning/gantts


In [3]:
!pip install -e ".[train]"

Obtaining file:///content/gdrive/My%20Drive/Voice_Cloning/gantts
     |████████████████████████████████| 1.7MB 9.3MB/s 
     |████████████████████████████████| 419kB 33.9MB/s 
     |████████████████████████████████| 368kB 23.2MB/s 
  Created wheel for nnmnkwii: filename=nnmnkwii-0.0.20-cp36-cp36m-linux_x86_64.whl size=1880506 sha256=201c19dbb2049b843d7d1671c85c11dfea2584ef12ea9ed202132dd65873ef97
  Stored in directory: /root/.cache/pip/wheels/c8/b2/6a/76aedd175231e475da518b6a63fcc6ecee567e57f29cbc1b05
  Created wheel for pysptk: filename=pysptk-0.1.18-cp36-cp36m-linux_x86_64.whl size=949538 sha256=feedd790af7201060a0a87c5554a33da7245d5cb7416dc63774fd13a4d1522eb
  Stored in directory: /root/.cache/pip/wheels/c7/96/d2/a163240019c59504402fab713af259026af81a99dea943404a
  Created wheel for bandmat: filename=bandmat-0.7-cp36-cp36m-linux_x86_64.whl size=898000 sha256=d4c9eb91b9258c9f4678dc81ae831af71f4cfc1bd29d62d18a59392a6c16b361
  Stored in directory: /root/.cache/pip/wheels/54/1e/28/429ff

In [2]:
%pylab inline
rcParams["figure.figsize"] = (16,5)

import sys
sys.path.insert(0, "..")

Populating the interactive namespace from numpy and matplotlib


In [3]:
import torch

from scipy.io import wavfile
import pyworld
import pysptk

import IPython
from IPython.display import Audio

import librosa
import librosa.display

from nnmnkwii import preprocessing as P
from glob import glob
from os.path import join

import gantts
from hparams import tts_acoustic as hp_acoustic
from hparams import tts_duration as hp_duration

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.preprocessing.data module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.preprocessing. Anything that cannot be imported from sklearn.preprocessing is now part of the private API.
  warnings.warn(message, FutureWarning)


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [8]:
!pip uninstall tensorflow

Uninstalling tensorflow-2.2.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.2.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.2.0


In [9]:
!pip install tensorflow==1.15

     |████████████████████████████████| 412.3MB 40kB/s 
     |████████████████████████████████| 3.8MB 34.1MB/s 
     |████████████████████████████████| 512kB 41.0MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=df905b227714ecda60de4e518e43994fe0424e398899fd367706581ea6d48e69
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3


In [6]:
!pip install pyworld

     |████████████████████████████████| 81kB 4.0MB/s 
  Created wheel for pyworld: filename=pyworld-0.2.8-cp36-cp36m-linux_x86_64.whl size=613105 sha256=711442a3372b1083e882937f47706bd87b6fd48e6df32f86e85a2f165e4be55b
  Stored in directory: /root/.cache/pip/wheels/c3/58/e5/a7e39ab92c56825f976970b97066753c68406c7fb0d80d4a4a
Successfully built pyworld


In [0]:
# Depends on experimental conditions
data_dir = "./data/cmu_arctic_tts_order24"
chechpoints_dir = "./checkpoints/tts_order24/"
duration_epoch = 100
acoustic_epoch = 50

In [5]:
X_min = {}
X_max = {}
Y_mean = {}
Y_var = {}
Y_std = {}

for typ in ["acoustic", "duration"]:
    X_min[typ] = np.load(join(data_dir, "X_{}_data_min.npy".format(typ)))
    X_max[typ] = np.load(join(data_dir, "X_{}_data_max.npy".format(typ)))
    Y_mean[typ] = np.load(join(data_dir, "Y_{}_data_mean.npy".format(typ)))
    Y_var[typ] = np.load(join(data_dir, "Y_{}_data_var.npy".format(typ)))
    Y_std[typ] = np.sqrt(Y_var[typ])

FileNotFoundError: ignored

In [0]:
models = {"baseline": {}, "gan": {}}

for method_typ in ["baseline", "gan"]:
    models[method_typ] = {"duration":  {}, "acoustic": {}}
    for (typ, epoch) in zip(["duration", "acoustic"], [duration_epoch, acoustic_epoch]):
        # Set missing hyper params from data
        hp = hp_acoustic if typ == "acoustic" else hp_duration
        if hp.generator_params["in_dim"] is None:
            hp.generator_params["in_dim"] = X_min[typ].shape[-1]
        if hp.generator_params["out_dim"] is None:
            hp.generator_params["out_dim"] = Y_mean[typ].shape[-1]
    
        models[method_typ][typ] = getattr(gantts.models, hp.generator)(**hp.generator_params)
        print("Model for {}, {}\n".format(method_typ, typ), models[method_typ][typ])
        
        checkpoint_path = join(chechpoints_dir, "tts_{}/{}/checkpoint_epoch{}_Generator.pth".format(
            typ, method_typ, epoch))
        print("Load checkpoint from: {}".format(checkpoint_path))
        
        checkpoint = torch.load(checkpoint_path)
        models[method_typ][typ].load_state_dict(checkpoint["state_dict"])
        models[method_typ][typ].eval()

In [0]:
from evaluation_tts import tts_from_label, get_lab_files, get_wav_files

In [0]:
label_dir = "../nnmnkwii_gallery/data/slt_arctic_full_data/label_state_align/"
wav_dir = "../nnmnkwii_gallery/data/slt_arctic_full_data/wav/"

test_label_paths = get_lab_files(data_dir,label_dir, test=True)
test_wav_paths = get_wav_files(data_dir, wav_dir, test=True)
print(len(test_label_paths))

In [0]:
for wav_path, label_path in zip(test_wav_paths, test_label_paths):
    print("Input label:", label_path)
    
    fs, waveform = wavfile.read(wav_path)
    
    ty = "baseline"
    baseline_waveform, mgc_baseline, _, _, _ = tts_from_label(
        models[ty], label_path, X_min, X_max, Y_mean, Y_std,
        apply_duration_model=False, post_filter=False)
    ty = "gan"
    gan_waveform, mgc_gan, _, _, _ = tts_from_label(
        models[ty], label_path, X_min, X_max, Y_mean, Y_std,
        apply_duration_model=False, post_filter=False)
    
    IPython.display.display(Audio(waveform, rate=fs))
    IPython.display.display(Audio(baseline_waveform, rate=fs))
    IPython.display.display(Audio(gan_waveform, rate=fs))

In [0]:
for wav_path, label_path in zip(test_wav_paths, test_label_paths):
    print("Input label:", label_path)
    
    fs, waveform = wavfile.read(wav_path)
    
    ty = "baseline"
    baseline_waveform, mgc_baseline, _, _, _ = tts_from_label(
        models[ty], label_path, X_min, X_max, Y_mean, Y_std,
        apply_duration_model=True, post_filter=False)
    ty = "gan"
    gan_waveform, mgc_gan, _, _, _ = tts_from_label(
        models[ty], label_path, X_min, X_max, Y_mean, Y_std,
        apply_duration_model=True, post_filter=False)
     
    IPython.display.display(Audio(waveform, rate=fs))
    IPython.display.display(Audio(baseline_waveform, rate=fs))
    IPython.display.display(Audio(gan_waveform, rate=fs))

In [0]:
def compute_mgc_from_waveform(x, fs):
    x = x.astype(np.float64)
    f0, timeaxis = pyworld.dio(x, fs, frame_period=hp_acoustic.frame_period)
    f0 = pyworld.stonemask(x, f0, timeaxis, fs)
    spectrogram = pyworld.cheaptrick(x, f0, timeaxis, fs)
    aperiodicity = pyworld.d4c(x, f0, timeaxis, fs)
    alpha = pysptk.util.mcepalpha(fs)
    mc = pysptk.sp2mc(spectrogram, order=hp_acoustic.order, alpha=alpha)   
    hop_length = int(fs * (hp_acoustic.frame_period * 0.001))
    modfs = fs / hop_length
    mc = P.modspec_smoothing(mc, modfs, cutoff=50)
    return mc

def vis_gv(label_path, wav_path):    
    fs, waveform = wavfile.read(wav_path)
    mgc_natural = compute_mgc_from_waveform(waveform, fs)

    waveform_baseline, mgc_baseline, _, _, _ = tts_from_label(
        models["baseline"], label_path, X_min, X_max, Y_mean, Y_std,
        apply_duration_model=True, post_filter=False)
    waveform_gan, mgc_gan, _, _, _ = tts_from_label(
        models["gan"], label_path, X_min, X_max, Y_mean, Y_std,
        apply_duration_model=True, post_filter=False)

    figure(figsize=(16,6))
    plot(np.var(mgc_natural, axis=0), "--", linewidth=2, label="Natural: global variances")
    plot(np.var(mgc_baseline, axis=0), linewidth=2, label="Baseline: global variances")
    plot(np.var(mgc_gan, axis=0), linewidth=2, label="GAN: global variances")
    legend(prop={"size": 18})
    yscale("log")
    xlabel("Dimention", fontsize=16)
        
    for x, name in [(waveform, "Natural"),
                   (waveform_baseline, "Baseline"),
                   (waveform_gan, "GAN")]:
        print(name)
        IPython.display.display(Audio(x, rate=fs))

In [0]:
idx = 0
label_path, wav_path = test_label_paths[idx], test_wav_paths[idx]
vis_gv(label_path, wav_path)

In [0]:
idx = 1
label_path, wav_path = test_label_paths[idx], test_wav_paths[idx]
vis_gv(label_path, wav_path)